# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b IemocapClass https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 166 (delta 49), reused 41 (delta 21), pack-reused 94 (from 3)
Receiving objects: 100% (166/166), 59.51 KiB | 9.92 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
RAVDESS pronto in cache: /kaggle/input/ravdess-emotional-speech-audio
Numero totale di file: 2880

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: dejolilandry/iemocapfullrelease...
100% 11.5G/11.5G [02:28<00:00, 83.7MB/s]
Extracting files...
✓ Dataset scaricato nella cache di sistema: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1
IEMOCAP pronto in cache: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1
Numero totale di file: 81249

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo
IEMOCAP: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [ ]:
from torch.utils.data import DataLoader
from dataset.custom_iemocap_dataset import CustomIEMOCAPDataset
import os

# Detect environment and set dataset path automatically
if os.path.exists('/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease'):
    # Running on Kaggle or local with kagglehub cache
    dataset_path = '/root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1'
    print("✓ Found IEMOCAP in kagglehub cache")
elif os.path.exists('/kaggle/input/iemocapfullrelease'):
    # Running on Kaggle - use cached dataset directly
    dataset_path = '/kaggle/input/iemocapfullrelease'
    print("✓ Detected Kaggle environment")
elif os.path.exists('/content/speech-emotion-recognition-25/iemocap/'):
    # Running on Colab
    dataset_path = '/content/speech-emotion-recognition-25/iemocap/'
    print("✓ Detected Colab environment")
else:
    # Running locally
    dataset_path = './data/'
    print("✓ Running locally")

print(f"📂 Using dataset path: {dataset_path}")

# Check if dataset exists (optional, since CustomIEMOCAPDataset will also check)
if not os.path.exists(dataset_path):
    print(f"⚠️ Warning: Dataset path not found: {dataset_path}")
    print("The dataset will be searched in the environment-specific location...")

# Create train dataset
print("\n🔄 Loading IEMOCAP dataset...")
train_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='train')
test_dataset = CustomIEMOCAPDataset(dataset_root=dataset_path, split='test')

print(f"\n✓ Train samples: {len(train_dataset)}")
print(f"✓ Test samples: {len(test_dataset)}")

# Create DataLoaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✓ DataLoaders created!")
print(f"  - Train batches: {len(train_dataloader)}")
print(f"  - Test batches: {len(test_dataloader)}")


✓ Running locally
📂 Using dataset path: ./data/

🔄 Loading IEMOCAP dataset...
📂 Starting to collect samples from: data

✅ Collected 0 samples in total


ValueError: No samples found in dataset!

In [ ]:
# Visualize training samples
from utils.visualization import visualize_samples, print_batch_info
print("Visualizing samples from training set...")
batch = next(iter(train_dataloader))
print_batch_info(batch)
print("\n")
visualize_samples(train_dataloader, num_samples=4)


ModuleNotFoundError: No module named 'utils.visualization'

# Step 4: Train your model and visualize training

In [ ]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [ ]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
#!python eval.py